# 硬件环境
- 100G硬盘

# 软件环境
- pytorch, ffmpeg, ffprobe

# 比赛读取文件路径如下

其中`query`和`refer`是比赛压缩包解压后的文件夹。
`query_frame`和`refer_frame`是抽取关键帧存储的文件夹。

In [3]:
import torch.nn as nn
import os
import sys
import glob
import shutil
import codecs
import subprocess

import pandas as pd
import numpy as np
import time

# %pylab inline
from PIL import Image
train_df = pd.read_csv('../input/train.csv')

from multiprocessing.pool import ThreadPool

In [4]:
query_id = glob.glob('./query_frame/*')
refer_id = glob.glob('./refer_frame/*')
# print(query_id)

In [5]:
# 读取query视频的关键帧，并按照视频和关键帧时间进行排序
query_imgs_path =glob.glob('./query_frame/*/*.jpg')
# for id in pd.read_csv('../input/submit_example.csv')['query_id']:
#     query_imgs_path += glob.glob('../input/query_frame/' + id + '/*.jpg')
query_imgs_path.sort(key=lambda x: x.lower())

In [6]:
# 读取refer视频的关键帧，并按照视频和关键帧时间进行排序

refer_imgs_path = glob.glob('./refer_frame/*/*.jpg')
refer_imgs_path.sort(key=lambda x: x.lower())
print(refer_imgs_path)

['./refer_frame/1244627000/1244627000_00000000.000000.jpg', './refer_frame/1244627000/1244627000_00000002.560000.jpg', './refer_frame/1244627000/1244627000_00000009.200000.jpg', './refer_frame/1244627000/1244627000_00000011.880000.jpg', './refer_frame/1244627000/1244627000_00000013.400000.jpg', './refer_frame/1244627000/1244627000_00000016.520000.jpg', './refer_frame/1244627000/1244627000_00000020.320000.jpg', './refer_frame/1244627000/1244627000_00000023.480000.jpg', './refer_frame/1244627000/1244627000_00000025.400000.jpg', './refer_frame/1244627000/1244627000_00000027.320000.jpg', './refer_frame/1244627000/1244627000_00000033.240000.jpg', './refer_frame/1244627000/1244627000_00000036.600000.jpg', './refer_frame/1244627000/1244627000_00000038.000000.jpg', './refer_frame/1244627000/1244627000_00000038.880000.jpg', './refer_frame/1244627000/1244627000_00000041.080000.jpg', './refer_frame/1244627000/1244627000_00000042.360000.jpg', './refer_frame/1244627000/1244627000_00000044.880000.jp

In [7]:
# import cv2
import imagehash
from PIL import Image
from tqdm import tqdm_notebook
from scipy.spatial.distance import cosine

def compute_cosin_distance(Q, feats, names):
    """
    feats and Q: L2-normalize, n*d
    """
    dists = np.dot(Q, feats.T)
#     print(dists)
    idxs = np.argsort(dists)[::-1]
#     print(idxs)
    rank_dists = dists[idxs]
#     print(rank_dists)
    rank_names = [names[k] for k in idxs]
#     print(rank_names)
    return idxs, rank_dists, rank_names

def hamming_distance(chaine1, chaine2):
    return sum(c1 != c2 for c1, c2 in zip(chaine1, chaine2))

In [8]:
class Net(nn.Module):
    def __init__(self, model):
        super(Net, self).__init__()
        # 取掉model的后1层
        self.resnet_layer = nn.Sequential(*list(model.children())[:-1])
        self.Linear_layer = nn.Linear(512, 4) #加上一层参数修改好的全连接层
    def forward(self, x):
        x = self.resnet_layer(x)
        x = x.view(x.size(0), -1)
        x = self.Linear_layer(x)
        return x

In [9]:
import torch
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data.dataset import Dataset

import logging
logging.basicConfig(level=logging.DEBUG, filename='example.log',
                    format='%(asctime)s - %(filename)s[line:%(lineno)d]: %(message)s')  # 

class QRDataset(Dataset):
    def __init__(self, img_path, transform=None):
        self.img_path = img_path
#         print(self.img_path)
        self.img_label = np.zeros(len(img_path))
    
        if transform is not None:
            self.transform = transform
        else:
            self.transform = None
    
    def __getitem__(self, index):
        img = Image.open(self.img_path[index])
        
        if self.transform is not None:
            img = self.transform(img)
        
        return img, self.img_path[index]

    def __len__(self):
        return len(self.img_path)
    
# model = models.resnet18(True).cuda()
# del model.fc
# model.fc=lambda x:x

model = torch.load('9_10000model.pth')
del model.Linear_layer
model.Linear_layer=lambda x:x
device = torch.device('cuda')  # 若能使用cuda，则使用cuda
model = model.to(device)

def predit_file(path):
    if not isinstance(path, list):
        path = [path]
    
    # print(path)
    
    data_loader = torch.utils.data.DataLoader(
        QRDataset(path, 
                transforms.Compose([
#                             transforms.Resize((256, 256)),  # 先调整图片大小至256x256
#                             transforms.RandomCrop((224, 224)),
                            transforms.Resize((224, 224)),
                            transforms.ToTensor(),
                            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])
        ), batch_size=40, shuffle=False, num_workers=0,
    )
    
    img_feat = []
    with torch.no_grad():
        for batch_data in tqdm_notebook(data_loader):
            batch_x, batch_y = batch_data
#             print(batch_x)
#             print(batch_y)
            
            # print(batch_y[:10])
            batch_x = Variable(batch_x).cuda()
            feat_pred = model(batch_x)
#             print(feat_pred)

            # max-pooling
            # feat_pred = F.max_pool2d(feat_pred, kernel_size=(24, 32))
            
            # ave-pooling
            # feat_pred = F.avg_pool2d(feat_pred, kernel_size=(24, 32))[:, :, 0, 0]
            
            #print(feat_pred.shape, batch_x.shape)
            feat_pred = feat_pred.data.cpu().numpy()
            # feat_pred = feat_pred.max(-1).max(-1)
            
            # feat_pred = feat_pred.reshape((-1, 512))
            img_feat.append(feat_pred)
            
            del feat_pred
            # img_feat.append(feat_pred)
            
    img_feat = np.vstack(img_feat)
    return img_feat

In [10]:
# 抽取query关键帧特征
query_cnn = predit_file(list(query_imgs_path[:]))
print(query_cnn[0])
# file = open('file_name.txt','w');

# file.write(str(list_variable));

# file.close();


[1.2381464  0.6935023  0.4520377  0.3241391  0.38500142 0.80131745
 0.5363313  0.08128363 0.2713597  0.29963404 0.38622245 1.0123856
 0.03398866 0.49128902 0.         1.6354637  0.11537884 0.74450994
 0.07020076 1.8264209  0.9110825  0.12883876 1.5581387  0.6323028
 0.73038435 0.416714   0.12771218 0.8843296  0.6986019  0.30843228
 0.3480945  0.69550246 0.3184215  2.0296803  0.39967373 0.22740869
 0.3217534  0.37244648 1.6947802  0.03953636 0.27463683 0.5940063
 1.2000754  1.6899383  0.99852407 1.1052446  0.06942884 1.9632488
 0.73866737 1.2376527  1.4435583  0.27528018 0.6262784  2.3784719
 0.5092599  0.107784   0.3594464  1.0075835  0.5265714  1.2123921
 0.57689774 1.6129522  2.0973277  0.46044788 1.3638815  0.5335308
 0.27279282 0.8790195  0.43017337 0.21855462 0.8098862  0.21189451
 0.3026117  1.7319164  0.8156301  0.6175959  1.3927914  1.9563863
 1.1474606  0.23434757 0.5311809  0.39307547 0.37123746 0.62789196
 1.987704   0.6513741  0.15304153 0.05225956 1.0395548  0.48316383
 0

In [11]:
# 抽取refer关键帧特征
refer_cnn = predit_file(list(refer_imgs_path[:]))
# file = open('refer_cnn.txt','w'

In [12]:
# -*- coding: utf-8 -*-
import os, sys, codecs
import glob

import numpy as np
# import cv2

from sklearn.preprocessing import normalize as sknormalize
from sklearn.decomposition import PCA

def normalize(x, copy=False):
    """
    A helper function that wraps the function of the same name in sklearn.
    This helper handles the case of a single column vector.
    """
    if type(x) == np.ndarray and len(x.shape) == 1:
        return np.squeeze(sknormalize(x.reshape(1,-1), copy=copy))
        #return np.squeeze(x / np.sqrt((x ** 2).sum(-1))[..., np.newaxis])
    else:
        return sknormalize(x, copy=copy)
        #return x / np.sqrt((x ** 2).sum(-1))[..., np.newaxis]

# 特征归一化
query_cnn = normalize(query_cnn)
refer_cnn = normalize(refer_cnn)

(791, 512)

(19692, 512)

# 计算相似关键帧

这一步只能计算得到关键帧相似结果，但具体的视频时间段还需要进一步对齐（TODO）。

In [60]:
import faiss


import numpy as np

d=int(query_cnn.shape[1])
nlist = 1                      #聚类中心的个数
k = 5
quantizer = faiss.IndexFlatL2(d)  # the other index
index = faiss.IndexIVFFlat(quantizer, d, nlist, faiss.METRIC_INNER_PRODUCT)

# assert not index.is_trained
index.train(refer_cnn)
# assert index.is_trained

index.add(refer_cnn)                  # add may be a bit slower as well
D1, I1 = index.search(query_cnn, k)     # actual search

index.nprobe = 10              # default nprobe is 1, try a few more
D2, I2 = index.search(query_cnn, k)


In [61]:
print(D1[6:10])
print(I1[6:10])
# print(D2[6:10])
# print(I2[6:10])
# print(query_imgs_path[200])
# # print(refer_imgs_path[11258])

# for i in I1[200]:
#     print(i)
#     print(refer_imgs_path[i])

# for i in I2[200]:
#     print(path.split('/')[-1], rank_names[i].split('/')[-1],rank_dists[i])
#     print(refer_imgs_path[i])
    
# for i in I1[791]:
#     print()
num=query_cnn.shape[0]
h=0
with open('faiss.txt','w+') as f:
    for i in range(num):
        for id,j in enumerate(I1[i]):
            if(D1[i][id]>0.8):
                print(query_imgs_path[i].split('/')[-1], refer_imgs_path[j].split('/')[-1],D1[i][id],file=f)

[[0.79223883 0.78834224 0.7864814  0.7860433  0.78526014]
 [0.7777432  0.76436687 0.7634167  0.7633834  0.76134217]
 [0.77676713 0.7745543  0.77180016 0.77042365 0.76748896]
 [0.78943837 0.78627694 0.7860211  0.7849425  0.7847029 ]]
[[ 5918  5882  5873 16872 16875]
 [10731  6540  6996  4017  3934]
 [ 6022  6185  2139  6183  2304]
 [ 9243  1855 11232 12199  4162]]


In [42]:
path_flag = [0]
with open("resnet18_90_norm_after7_train.txt",'w') as f:
    for idx, (path, feat) in tqdm_notebook(enumerate(zip(query_imgs_path, query_cnn))):
#         print(path)
        idxs, rank_dists, rank_names = compute_cosin_distance(feat, refer_cnn, refer_imgs_path)
#         print(rank_names[0])
#         if path.split('/')[-1]=='21443adc-b911-11e9-ad99-fa163ee49799_00000075.800000.jpg':
#             print(feat)
#             print(path.split('/')[-1], rank_names[0].split('/')[-1],rank_dists[0])
#         if rank_dists[0] > 0.9:
        # if hamming_distance(refer_hash[idxs[0]], query_hash[idx]) < 5:
        
#             if path.split('/')[-2] != path_flag[-1]:
#             print('')
        for i in range(5):
            if(rank_dists[i]>0.8):
                print(path.split('/')[-1], rank_names[i].split('/')[-1],rank_dists[i],file=f)
                print(path.split('/')[-1], rank_names[i].split('/')[-1],rank_dists[i])
                path_flag.append(path.split('/')[-2])

21443adc-b911-11e9-ad99-fa163ee49799_00000000.000000.jpg 2620315400_00002298.560000.jpg 0.8066108
21443adc-b911-11e9-ad99-fa163ee49799_00000000.000000.jpg 1530507600_00003608.000000.jpg 0.8046155
21443adc-b911-11e9-ad99-fa163ee49799_00000000.000000.jpg 1530507600_00000052.920000.jpg 0.8015628
21443adc-b911-11e9-ad99-fa163ee49799_00000053.400000.jpg 1753812600_00001588.400000.jpg 0.80925095
21443adc-b911-11e9-ad99-fa163ee49799_00000053.400000.jpg 1376903200_00002804.840000.jpg 0.8065695
21443adc-b911-11e9-ad99-fa163ee49799_00000053.400000.jpg 1629260900_00003008.000000.jpg 0.8044797
21443adc-b911-11e9-ad99-fa163ee49799_00000053.400000.jpg 1629260900_00005026.200000.jpg 0.8029431
21443adc-b911-11e9-ad99-fa163ee49799_00000063.866667.jpg 2620315400_00002551.240000.jpg 0.8001975
21443adc-b911-11e9-ad99-fa163ee49799_00000065.333333.jpg 3043930400_00003396.400000.jpg 0.83308315
21443adc-b911-11e9-ad99-fa163ee49799_00000065.333333.jpg 2620315400_00005297.560000.jpg 0.8266616
21443adc-b911-11e9

31023ca2-b994-11e9-8b81-fa163ee49799_00000023.000000.jpg 2927857700_00004906.800000.jpg 0.82082254
31023ca2-b994-11e9-8b81-fa163ee49799_00000024.733333.jpg 1629260900_00004146.960000.jpg 0.80242634
31023ca2-b994-11e9-8b81-fa163ee49799_00000028.600000.jpg 1629260900_00004443.400000.jpg 0.80796987
31023ca2-b994-11e9-8b81-fa163ee49799_00000028.600000.jpg 1398481500_00002864.280000.jpg 0.8002753
31023ca2-b994-11e9-8b81-fa163ee49799_00000031.400000.jpg 1596046700_00004953.240000.jpg 0.80403227
31023ca2-b994-11e9-8b81-fa163ee49799_00000055.200000.jpg 1596046700_00005125.440000.jpg 0.8055167
31023ca2-b994-11e9-8b81-fa163ee49799_00000055.200000.jpg 1723849300_00001955.320000.jpg 0.8052621
31023ca2-b994-11e9-8b81-fa163ee49799_00000055.200000.jpg 1773351300_00001331.000000.jpg 0.8007896
31023ca2-b994-11e9-8b81-fa163ee49799_00000055.600000.jpg 1753812600_00000061.480000.jpg 0.8150506
31023ca2-b994-11e9-8b81-fa163ee49799_00000055.600000.jpg 3043930400_00006241.960000.jpg 0.8144398
31023ca2-b994-11

374a3a0c-b93e-11e9-8b81-fa163ee49799_00000013.300000.jpg 1753812600_00002677.600000.jpg 0.8278003
374a3a0c-b93e-11e9-8b81-fa163ee49799_00000013.300000.jpg 1596046700_00002606.040000.jpg 0.8129211
374a3a0c-b93e-11e9-8b81-fa163ee49799_00000013.300000.jpg 2400411900_00000993.800000.jpg 0.8090346
374a3a0c-b93e-11e9-8b81-fa163ee49799_00000013.300000.jpg 1596046700_00000995.640000.jpg 0.80834484
374a3a0c-b93e-11e9-8b81-fa163ee49799_00000013.300000.jpg 2620315400_00006036.400000.jpg 0.80645585
374a3a0c-b93e-11e9-8b81-fa163ee49799_00000024.200000.jpg 1782169900_00004563.680000.jpg 0.8030414
374a3a0c-b93e-11e9-8b81-fa163ee49799_00000027.000000.jpg 1782169900_00003481.160000.jpg 0.8043743
374a3a0c-b93e-11e9-8b81-fa163ee49799_00000038.600000.jpg 2927857700_00000273.000000.jpg 0.81574506
374a3a0c-b93e-11e9-8b81-fa163ee49799_00000038.600000.jpg 1629260900_00004465.520000.jpg 0.8080282
374a3a0c-b93e-11e9-8b81-fa163ee49799_00000038.600000.jpg 1629260900_00004536.840000.jpg 0.80374753
374a3a0c-b93e-11

3e840a16-b964-11e9-8b81-fa163ee49799_00000014.466667.jpg 1629260900_00004217.920000.jpg 0.8494579
3e840a16-b964-11e9-8b81-fa163ee49799_00000014.466667.jpg 1376903200_00003870.200000.jpg 0.8439338
3e840a16-b964-11e9-8b81-fa163ee49799_00000014.466667.jpg 1629260900_00004412.160000.jpg 0.8415226
3e840a16-b964-11e9-8b81-fa163ee49799_00000014.466667.jpg 1376903200_00003053.200000.jpg 0.83905625
3e840a16-b964-11e9-8b81-fa163ee49799_00000014.466667.jpg 1629260900_00004180.920000.jpg 0.8368192
3e840a16-b964-11e9-8b81-fa163ee49799_00000016.266667.jpg 1629260900_00005115.200000.jpg 0.8145038
3e840a16-b964-11e9-8b81-fa163ee49799_00000016.266667.jpg 1629260900_00004088.440000.jpg 0.80630064
3e840a16-b964-11e9-8b81-fa163ee49799_00000016.266667.jpg 1629260900_00000845.400000.jpg 0.80565846
3e840a16-b964-11e9-8b81-fa163ee49799_00000016.266667.jpg 2927857700_00005415.960000.jpg 0.8029959
3e840a16-b964-11e9-8b81-fa163ee49799_00000016.266667.jpg 3043930400_00001762.680000.jpg 0.8014814
3e840a16-b964-11e

50f7e5ec-b986-11e9-9c9e-fa163e3d9e3c_00000050.466667.jpg 2333805400_00003616.960000.jpg 0.8621266
50f7e5ec-b986-11e9-9c9e-fa163e3d9e3c_00000050.466667.jpg 2333805400_00000691.000000.jpg 0.85813195
50f7e5ec-b986-11e9-9c9e-fa163e3d9e3c_00000050.466667.jpg 2333805400_00002707.360000.jpg 0.8569697
50f7e5ec-b986-11e9-9c9e-fa163e3d9e3c_00000050.466667.jpg 2333805400_00002726.240000.jpg 0.8552337
50f7e5ec-b986-11e9-9c9e-fa163e3d9e3c_00000050.466667.jpg 2333805400_00000051.680000.jpg 0.8529619
50f7e5ec-b986-11e9-9c9e-fa163e3d9e3c_00000052.466667.jpg 2333805400_00003860.560000.jpg 0.8487742
50f7e5ec-b986-11e9-9c9e-fa163e3d9e3c_00000052.466667.jpg 2333805400_00003869.080000.jpg 0.8336558
50f7e5ec-b986-11e9-9c9e-fa163e3d9e3c_00000052.466667.jpg 2333805400_00004540.000000.jpg 0.8321372
50f7e5ec-b986-11e9-9c9e-fa163e3d9e3c_00000052.466667.jpg 2333805400_00003850.560000.jpg 0.82985103
50f7e5ec-b986-11e9-9c9e-fa163e3d9e3c_00000052.466667.jpg 1629260900_00004170.920000.jpg 0.8292831
50f7e5ec-b986-11e9

...

b62a8b88-b8cb-11e9-930e-fa163ee49799_00000018.000000.jpg 1244627000_00000719.960000.jpg 0.8750922
b62a8b88-b8cb-11e9-930e-fa163ee49799_00000018.000000.jpg 1244627000_00001007.920000.jpg 0.8524427
b62a8b88-b8cb-11e9-930e-fa163ee49799_00000018.000000.jpg 1244627000_00000437.440000.jpg 0.82152647
b62a8b88-b8cb-11e9-930e-fa163ee49799_00000018.000000.jpg 1376903200_00000813.560000.jpg 0.80942094
b62a8b88-b8cb-11e9-930e-fa163ee49799_00000018.000000.jpg 1376903200_00000813.720000.jpg 0.8081943
b62a8b88-b8cb-11e9-930e-fa163ee49799_00000019.200000.jpg 1244627000_00000721.400000.jpg 0.88168174
b62a8b88-b8cb-11e9-930e-fa163ee49799_00000019.200000.jpg 1244627000_00000735.480000.jpg 0.85637176
b62a8b88-b8cb-11e9-930e-fa163ee49799_00000019.200000.jpg 1244627000_00000738.160000.jpg 0.82393974
b62a8b88-b8cb-11e9-930e-fa163ee49799_00000019.200000.jpg 3043930400_00005586.120000.jpg 0.82102287
b62a8b88-b8cb-11e9-930e-fa163ee49799_00000019.200000.jpg 1782169900_00005713.320000.jpg 0.8143151
b62a8b88-b8cb-

c008e798-b98f-11e9-8b81-fa163ee49799_00000016.300000.jpg 1596046700_00002921.040000.jpg 0.8960398
c008e798-b98f-11e9-8b81-fa163ee49799_00000016.300000.jpg 1596046700_00002946.840000.jpg 0.89522004
c008e798-b98f-11e9-8b81-fa163ee49799_00000016.300000.jpg 1596046700_00004051.680000.jpg 0.87522864
c008e798-b98f-11e9-8b81-fa163ee49799_00000016.300000.jpg 1596046700_00003778.040000.jpg 0.8737229
c008e798-b98f-11e9-8b81-fa163ee49799_00000016.300000.jpg 1596046700_00003048.480000.jpg 0.87073183
c008e798-b98f-11e9-8b81-fa163ee49799_00000026.500000.jpg 1596046700_00002959.280000.jpg 0.8562068
c008e798-b98f-11e9-8b81-fa163ee49799_00000026.500000.jpg 1596046700_00005226.680000.jpg 0.843551
c008e798-b98f-11e9-8b81-fa163ee49799_00000026.500000.jpg 1596046700_00003076.600000.jpg 0.8414295
c008e798-b98f-11e9-8b81-fa163ee49799_00000026.500000.jpg 1596046700_00002996.920000.jpg 0.82604563
c008e798-b98f-11e9-8b81-fa163ee49799_00000026.500000.jpg 1596046700_00003352.440000.jpg 0.8240241
c008e798-b98f-11e

d880856a-b9a8-11e9-b8ed-fa163e063a2b_00000053.133333.jpg 1804312000_00005508.160000.jpg 0.80677044
d880856a-b9a8-11e9-b8ed-fa163e063a2b_00000054.466667.jpg 1804312000_00005493.600000.jpg 0.8534737
d880856a-b9a8-11e9-b8ed-fa163e063a2b_00000054.466667.jpg 1804312000_00002950.560000.jpg 0.84699106
d880856a-b9a8-11e9-b8ed-fa163e063a2b_00000054.466667.jpg 1804312000_00005408.040000.jpg 0.8180677
d880856a-b9a8-11e9-b8ed-fa163e063a2b_00000054.466667.jpg 1804312000_00005595.120000.jpg 0.8145729
d880856a-b9a8-11e9-b8ed-fa163e063a2b_00000057.333333.jpg 3043930400_00007570.360000.jpg 0.8025283
d880856a-b9a8-11e9-b8ed-fa163e063a2b_00000060.666667.jpg 1773351300_00001298.520000.jpg 0.80892026
d880856a-b9a8-11e9-b8ed-fa163e063a2b_00000060.666667.jpg 1782169900_00005681.480000.jpg 0.8012588
d880856a-b9a8-11e9-b8ed-fa163e063a2b_00000060.666667.jpg 1773351300_00001326.320000.jpg 0.80014884
d880856a-b9a8-11e9-b8ed-fa163e063a2b_00000062.333333.jpg 1804312000_00000548.800000.jpg 0.8035883
d880856a-b9a8-11

e2f1ea86-b92e-11e9-9c9e-fa163e3d9e3c_00000041.600000.jpg 1753812600_00002152.680000.jpg 0.82492346
e2f1ea86-b92e-11e9-9c9e-fa163e3d9e3c_00000041.600000.jpg 1753812600_00005222.120000.jpg 0.8139368
e2f1ea86-b92e-11e9-9c9e-fa163e3d9e3c_00000041.600000.jpg 3043930400_00004579.400000.jpg 0.81077313
e2f1ea86-b92e-11e9-9c9e-fa163e3d9e3c_00000041.600000.jpg 1723849300_00003939.360000.jpg 0.8077568
e2f1ea86-b92e-11e9-9c9e-fa163e3d9e3c_00000041.600000.jpg 2333805400_00000456.600000.jpg 0.80689
e2f1ea86-b92e-11e9-9c9e-fa163e3d9e3c_00000042.700000.jpg 1723849300_00004306.880000.jpg 0.84303474
e2f1ea86-b92e-11e9-9c9e-fa163e3d9e3c_00000042.700000.jpg 1723849300_00004316.880000.jpg 0.8388985
e2f1ea86-b92e-11e9-9c9e-fa163e3d9e3c_00000042.700000.jpg 1723849300_00003594.240000.jpg 0.8312985
e2f1ea86-b92e-11e9-9c9e-fa163e3d9e3c_00000042.700000.jpg 1723849300_00003838.840000.jpg 0.82183754
e2f1ea86-b92e-11e9-9c9e-fa163e3d9e3c_00000042.700000.jpg 1723849300_00003829.440000.jpg 0.8184651
e2f1ea86-b92e-11e9

e2f1ea86-b92e-11e9-9c9e-fa163e3d9e3c_00000146.200000.jpg 1782169900_00006119.160000.jpg 0.82372594
e2f1ea86-b92e-11e9-9c9e-fa163e3d9e3c_00000146.200000.jpg 1629260900_00004303.640000.jpg 0.8214216
e2f1ea86-b92e-11e9-9c9e-fa163e3d9e3c_00000146.200000.jpg 2400411900_00006734.440000.jpg 0.81858325
e2f1ea86-b92e-11e9-9c9e-fa163e3d9e3c_00000146.200000.jpg 1629260900_00004443.400000.jpg 0.8140923
e2f1ea86-b92e-11e9-9c9e-fa163e3d9e3c_00000146.200000.jpg 1782169900_00000684.120000.jpg 0.81259334
f5006a56-b922-11e9-8b81-fa163ee49799_00000008.600000.jpg 2927857700_00005327.480000.jpg 0.8353121
f5006a56-b922-11e9-8b81-fa163ee49799_00000008.600000.jpg 2620315400_00004943.680000.jpg 0.82776976
f5006a56-b922-11e9-8b81-fa163ee49799_00000008.600000.jpg 3043930400_00002511.560000.jpg 0.8223158
f5006a56-b922-11e9-8b81-fa163ee49799_00000008.600000.jpg 3043930400_00003526.560000.jpg 0.81685174
f5006a56-b922-11e9-8b81-fa163ee49799_00000008.600000.jpg 2927857700_00002737.880000.jpg 0.8151049
f5006a56-b922-1